In [2]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels
from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing
import seaborn as sns
from scipy import optimize
import time
import functools
from multiprocessing import Pool
#from cyipopt import minimize_ipopt
from numpy.polynomial.polynomial import polyfit
import matplotlib.pyplot as plt
import random

import warnings
warnings.filterwarnings("ignore")

random.seed(10)


In [3]:
input_dir = r"/Users/juju/Library/Mobile Documents/com~apple~CloudDocs/ta_project/output/welfare/fixed_price"
input_dir_eq = r"/Users/juju/Library/Mobile Documents/com~apple~CloudDocs/ta_project/src"

In [4]:
data = pd.read_csv(os.path.join(input_dir,r"welfare_fixed_price_result_const_new.csv"))

data_eq = pd.read_csv(os.path.join(input_dir_eq,r"df_ne.csv"))

In [5]:
list(data_eq.columns)

['index',
 'post_month',
 'shareid',
 'year_month',
 'Supply',
 'Demand',
 'Revenue',
 'Occ',
 'ADR',
 'RevPAR',
 'Country',
 'Market',
 'Chain',
 'Owner',
 'Mgmt',
 'ParentCompany',
 'Operation',
 'Scale',
 'Class',
 'Location',
 'OpenDate',
 'SizeCode',
 'max_str_month',
 'max_review_month',
 'min_review_month',
 'on_ta',
 'monthly_rating_mean',
 'monthly_review_count',
 'monthly_one_star_count',
 'monthly_two_star_count',
 'monthly_three_star_count',
 'monthly_four_star_count',
 'monthly_five_star_count',
 'monthly_hotel_response_count',
 'monthly_partnership_reviews_count',
 'cleanliness_count',
 'cleanliness_mean',
 'location_count',
 'location_mean',
 'sleep_quality_count',
 'sleep_quality_mean',
 'room_quality_count',
 'room_quality_mean',
 'service_count',
 'service_mean',
 'value_count',
 'value_mean',
 'renovate',
 'solo_count',
 'couple_count',
 'family_count',
 'business_count',
 'month_with_reviews',
 'num_of_reviews',
 'total_one_star',
 'total_two_star',
 'total_three_st

In [8]:
data["Supply"].sum()

65054.53479452054

In [90]:

alpha = -0.012
sigma = 0.5396
rating_param = 0.073

occ_upper= 88


In [91]:

data = data.merge(data_eq[["q_upper","hotel_fixed_effect","market_time_fixed_effect",\
                          "shareid","year","month","max_group_price","market_size"]], on=["shareid","year","month"])


In [92]:
supply_max = data.groupby(["shareid","year"])["Supply"].max().reset_index().rename(columns={"Supply":"max_supply_year"})
data= data.merge(supply_max, on=["shareid","year"], how="left")
data["capacity"] = data["max_supply_year"]/30

# Calculate marginal cost and total cost

In [93]:
data_lower = data[data["Demand"]<=data["q_upper"]]

data_upper = data[data["Demand"]>data["q_upper"]]

data_lower["excess_demand"] = 0
data_lower["cost_ne"] = (192.39814102368837+data_lower["hotel_fixed_effect"]+data_lower["market_time_fixed_effect"])*\
                           data_lower["Demand"]
data_upper["excess_demand"] = data_upper["Demand"]-data_upper["q_upper"]
data_upper["cost_ne"] = (192.39814102368837+data_upper["hotel_fixed_effect"]+data_upper["market_time_fixed_effect"])*\
                           data_upper["q_upper"]+0.5*0.043841100663268406*(data_upper["excess_demand"]**2)


data= pd.concat([data_lower,data_upper],axis=0)

data = data.fillna(0)



In [94]:
data

,level_0,index,post_month,shareid,year_month,Supply,Demand,Revenue,Occ,ADR,...,D_g_sum,q_upper,hotel_fixed_effect,market_time_fixed_effect,max_group_price,market_size,max_supply_year,capacity,excess_demand,cost_ne
0,0,0,2016-08,75909,2016-08-01,2170.0,1464.0,209160.0,67.4654,142.868852,...,2.542960,1909.60,0.000000,-119.170953,142.868852,325922.6,2170.0,72.333333,0.0,1.072046e+05
1,1,1,2016-09,75909,2016-09-01,2100.0,700.0,75000.0,33.3333,107.142857,...,1.565520,1848.00,0.000000,-126.749510,148.049200,325922.6,2170.0,72.333333,0.0,4.595404e+04
2,2,2,2016-10,75909,2016-10-01,2170.0,1123.0,123965.0,51.7511,110.387355,...,1.649075,1909.60,0.000000,-103.913689,162.612529,325922.6,2170.0,72.333333,0.0,9.936804e+04
3,3,3,2016-11,75909,2016-11-01,1956.0,1282.0,201327.0,65.5419,157.041342,...,2.067547,1721.28,0.000000,-74.653972,166.673548,325922.6,2170.0,72.333333,0.0,1.509480e+05
4,4,4,2016-12,75909,2016-12-01,2003.0,1020.0,199453.0,50.9236,195.542157,...,2.220051,1762.64,0.000000,13.083626,212.543127,325922.6,2170.0,72.333333,0.0,2.095914e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5588,73087,17671,2016-06,9990853,2016-06-01,12450.0,11596.0,3328976.0,93.1405,287.079683,...,5.716251,10956.00,-95.081347,100.209591,514.487154,1210161.0,12865.0,428.833333,640.0,2.173078e+06
5589,73088,17672,2016-07,9990853,2016-07-01,12865.0,11963.0,2738027.0,92.9887,228.874613,...,6.719439,11321.20,-95.081347,69.794834,460.997903,1210161.0,12865.0,428.833333,641.8,1.900933e+06
5590,73089,17673,2016-08,9990853,2016-08-01,12865.0,11545.0,2432220.0,89.7396,210.673019,...,4.997755,11321.20,-95.081347,57.166410,433.492404,1210161.0,12865.0,428.833333,223.8,1.750033e+06
5591,73090,17674,2016-09,9990853,2016-09-01,12450.0,11283.0,2880734.0,90.6265,255.316317,...,4.827411,10956.00,-95.081347,96.538286,503.537241,1210161.0,12865.0,428.833333,327.0,2.126220e+06


In [95]:
# Counterfactual marginal cost

data["demand_cf"] = data["market_share_cf"]*data["market_size"]

data_lower = data[data["demand_cf"]<=data["q_upper"]]
data_upper = data[data["demand_cf"]>data["q_upper"]]

data_lower["excess_demand"] = 0
data_lower["cost_cf"] = (192.39814102368837+data_lower["hotel_fixed_effect"]+data_lower["market_time_fixed_effect"])*\
                           data_lower["demand_cf"]
data_upper["excess_demand"] = data_upper["demand_cf"]-data_upper["q_upper"]
data_upper["cost_cf"] = (192.39814102368837+data_upper["hotel_fixed_effect"]+data_upper["market_time_fixed_effect"])*\
                           data_upper["q_upper"]+0.5*0.043841100663268406*(data_upper["excess_demand"]**2)

data = pd.concat([data_lower,data_upper],axis=0)



# Calculate Consumer Surplus

In [96]:
# calculate walfare
data["market_share_cf_delta_diff"] = data["market_share_cf"]*rating_param*(data["Rating"] - data["exante_rating"])

l=[]
for i in (0,1,2):
    
    df = data[data["Market"]==data["Market"].unique()[i]]
    diff = df.groupby(["month","year"])["market_share_cf_delta_diff"].sum().reset_index().rename(columns={"market_share_cf_delta_diff":"diff_term"})
    temp = df.merge(diff,on=["month","year"], how="left")
    l.append(temp)
    
df = pd.concat(l,axis=0)


In [97]:
df["r_greater_exante"] = df["exante_rating"]<df["Rating"]

#df["market_share_percent_diff"] = (df["market_share_cf"] - df["market_share"])/df["market_share"]


In [98]:
df_cs = df[["Market","year","month","Market Monthly Supply","D_g_sum","D_g_sum_cf","diff_term","market_size"]].drop_duplicates()

df_cs["cs"] = (np.log(1+df_cs["D_g_sum"]) - np.log(1+df_cs["D_g_sum_cf"])-df_cs["diff_term"])/alpha



In [99]:
df_cs["month_cs"] = df_cs["cs"]*df_cs["market_size"]

cs_result = df_cs.groupby("Market")["month_cs"].sum()/1000000
cs_result = cs_result.reset_index("Market")
cs_result.transpose()


,0,1,2
Market,"Chicago, IL","Houston, TX","Miami/Hialeah, FL"
month_cs,-0.570724,-1.882986,-0.202125


# Calculate Revenue, Profit, Marginal Cost

In [100]:
df["Revenue_cf"] = df["market_share_cf"]*df["market_size"]*df["ADR"]
df["revenue_diff"] = df["Revenue_cf"] - df["Revenue"]
revenue_result = df.groupby("Market")["revenue_diff"].sum()/1000000
revenue_result = revenue_result.reset_index("Market")
revenue_result.transpose()


,0,1,2
Market,"Chicago, IL","Houston, TX","Miami/Hialeah, FL"
revenue_diff,-24.038875,-21.781828,-12.253758


In [101]:
revenue_result["revenue_diff"].sum()

-58.07446032758657

In [102]:
revenue_breakdown_result = df.groupby(["Market","r_greater_exante"])["revenue_diff"].sum()/1000000
revenue_breakdown_result.reset_index().transpose()                                      

,0,1,2,3,4,5
Market,"Chicago, IL","Chicago, IL","Houston, TX","Houston, TX","Miami/Hialeah, FL","Miami/Hialeah, FL"
r_greater_exante,False,True,False,True,False,True
revenue_diff,17.61694,-41.655815,19.792079,-41.573907,8.911319,-21.165077


In [103]:
df.groupby(["r_greater_exante"])["revenue_diff"].sum()/1000000

r_greater_exante
False     46.320338
True    -104.394798
Name: revenue_diff, dtype: float64

In [104]:
df["ps_cf"] = df["Revenue_cf"]-df["cost_cf"]
df["ps_ne"] = df["Revenue"]-df["cost_ne"]
df["ps_diff"] = df["ps_cf"] - df["ps_ne"]
df.groupby("Market")["ps_diff"].sum()/1000000

Market
Chicago, IL         -1.493540
Houston, TX         -1.721582
Miami/Hialeah, FL    1.651945
Name: ps_diff, dtype: float64

In [105]:
df["ps_diff"].sum()/1000000

-1.5631767309705102

In [106]:
df.groupby(["r_greater_exante"])["ps_diff"].sum()/1000000

r_greater_exante
False    21.365039
True    -22.928216
Name: ps_diff, dtype: float64

In [107]:
df["cost_diff"] = df["cost_cf"] - df["cost_ne"]
df["cost_diff"].sum()/1000000



-56.51128359661606

In [108]:
df.groupby(["r_greater_exante"])["cost_diff"].sum()/1000000

r_greater_exante
False    24.955299
True    -81.466583
Name: cost_diff, dtype: float64

In [109]:
df.groupby("Market")["cost_diff"].sum()/1000000

Market
Chicago, IL         -22.545335
Houston, TX         -20.060245
Miami/Hialeah, FL   -13.905703
Name: cost_diff, dtype: float64

In [110]:
total_market_share = df.groupby(["Market","year","month"])["market_share"].sum().reset_index().rename(columns={"market_share":"market_share_sum"})
total_market_share_cf = df.groupby(["Market","year","month"])["market_share_cf"].sum().reset_index().rename(columns={"market_share_cf":"market_share_sum_cf"})

df_2 = df.merge(total_market_share, on=["Market","year","month"]).merge(total_market_share_cf, on=["Market","year","month"])
                                                                                                           



In [111]:
df_2["market_share_sum_diff"] = df_2["market_share_sum_cf"] - df_2["market_share_sum"]

total_market_share = df_2[["Market","year","month","market_share_sum_diff","market_size"]].drop_duplicates()

total_market_share["market_share_sum_diff"].mean()*100

-0.30804645290566524

In [112]:

total_market_share.groupby("Market")["market_share_sum_diff"].mean()*100

Market
Chicago, IL         -0.438947
Houston, TX         -0.104045
Miami/Hialeah, FL   -0.381148
Name: market_share_sum_diff, dtype: float64

In [113]:
df_2.groupby("Market")["market_size"].unique()

Market
Chicago, IL          [1210161.0]
Houston, TX          [1419322.0]
Miami/Hialeah, FL     [325922.6]
Name: market_size, dtype: object

In [1]:
(325923+1419322+1210161)*12

35464872

In [211]:
df.groupby(["Market","Operation"])["capacity"]